 # Aprendizaje Automático : Trabajo Práctico 2


# Integrantes: Juan Manuel Perez, Mariela Rajngewerc, Tomás Freilij

# Resumen:

Para el presente trabajo implementamos el algoritmo Q-Learning para la resolución del juego Cuatro en linea. El objetivo será mostrar cómo un jugador guiado por el algoritmo en cuestión termina superando ampliamente a uno que elija todos sus movimientos al azar.



# Palabras clave: 

aprendizaje por refuerzos, Q-Learning, cuatro en linea, 


# Desarrollo

El trabajo consistió en armar una implementación del cuatro en línea que nos permitiera simular fielmente el desarrollo de una partida. Para eso primero lo armamos para que puedan jugar dos humanos.

En una segunda etapa, agregamos el jugador random (lo cual no presentó muchas dificultades) y un Qplayer que va a jugar guiado por el algoritmo en cuestión.

Nuestra hipotesis es que cuanto más entrenamiento realicemos sobre el QPLayer, la tabla Q sera actualizada de forma más eficiente y así, dado un estado y una acción, el QPlayer sabrá cuan conveniente es tomar esa acción.

Al momento de elegir una jugada decidimos no usar siempre la experiencia que va adquiriendo el QPlayer sino, con probabilidad epsilon, elegir alguna accion posible al azar. La idea es que el jugador no juegue solo con las acciones conocidas y que pruebe otros caminos.



# Estados

Los estados están definidos por la configuración de fichas de un tablero. 
Pensamos definir algún tipo de propiedad más genérica que englobe a muchos momentos de un tablero en un mismo estado. Por ejemplo, en una partida muy avanzada podría no ser necesario analizar las fichas que están en el fondo del tablero, ya que no existe posibilidad de ganar o perder por cómo están dispuestas las mismas.

Esto último nos pareció que traería complicaciones implementativas y que comparar dos tableros casilla a casilla no debería ser problemático. Entonces priorizamos una implementación más fácil, por sobre un mejor uso de la memoria o cómputo más eficiente.



# Tabla Q

Por lo dicho anterior, nuestra tabla Q será un mapeo de tuplas (estado, jugada) -> valor.

Si tenemos en cuenta que dado un estado, a lo sumo existen siete posibles jugadas, el tamaño de la tabla tendrá un tamaño de O(n), donde n es la cantidad de estados posibles. Como además existen estados inalcanzables, la tabla no será precisamente de esos órdenes de tamaño.

Además, la tabla Q no la inicializamos en su totalidad desde el comienzo sino que se hará bajo demanda. En caso de que no se tenga un valor definido para la tabla, se tomará alguno por defecto (quizás al azar) a la hora de los cálculos.

De esta forma, la tabla irá creciendo con el tiempo.




# Estrategia adoptada 

Como en el ajedrez, en este juego existe la posibilidad de, antes de hacer un movimiento, explorar las posibles respuestas al mismo. Así tendríamos un mejor análisis de qué jugar, o de en qué situación estamos (si perdiendo, ganando o empatando).

Por ejemplo, si estamos en un estado del juego en el que una jugada nos lleva a perder de forma obligada, es evidente que no optaríamos por la misma. A su vez, si una nos lleva a ganar de forma directa, tenemos que ir por ese camino.

Es de esta forma que nuestro Qplayer, haciendo este tipo de análisis, irá aprendiendo. En este caso, aprender es equivalente a aumentar la probabilidad de ganar en una futura partida.

La profundidad de análisis (cuántos movimientos a futuro analiza) será algo parametrizable en el Q-Player. En los hechos, resultó que una profundidad de seis o más movimientos a futuro lleva a que cada partida dure alrededor de una hora. Por eso mismo, los análisis serán analizando dos o cuatro.



# Pseudocódigo de "encontrar mejor jugada"

La función central está en la elección del mejor movimiento para un estado dado.
Para eso, QPlayer tiene implementado "mejor_accion_para" que recibe un estado del juego y se encarga de encontrar la mejor respuesta.

Veamos el pseudocódigo de la misma:



Si número al azar < épsilon

    mejor_jugada = elegir jugada al azar
    
    
Si no

    Para cada acción posible J del estado actual
        Si el valor de Q(estado,J) mejora el de Q(estado,mejor_jugada)
            mejor_valor = Q(estado,mejor_jugada)
            mejor_jugada = J
        
       
viejo_valor = obtener_valor_en_tabla_q(estado,mejor_jugada)

Q(estado,mejor_jugada)] = viejo_valor + alpha * ( recompensa(estado,mejor_jugada) + gamma * mejor_valor - viejo_valor )
        

# Pseudocódigo de "encontrar valor q"

Dado un estado S y una posible jugada J:

Si jugando J gano
    devolver 100
Si jugando J gana mi rival
    devolver -100 
Si jugando J hay empate 
    devolver 0
    
Si no
    Si llegué a la profunidad de análisis máxima
       Para cada acción posibles del rival encontrar su mejor jugada
           devolver el valor para esa jugada del rival
           
    Si queda por recorrer más profundidad
    
        llamar recursivamente a Q para cada acción mía posibles y quedarme con la mejor




# Cálculo de las Recompensa

La funcion recompensa le adjudica a cada (estado, accion) un número. 
Representa el peso que tiene la casilla donde el QPlayer pondrá ficha (osea, la accion que ejercerá). 
Decidimos considerar los siguientes resultados posibles:

- recompensa_max: en el caso de que el QPlayer resulte ganador

- recompensa_media: en el caso que ya hubiera 2 fichas concecutivas del Qplayer, se pueda agregar una tercera y haya un lugar libre donde, en jugadas posteriores podria completarse el cuatro en línea.

- recompensa_baja: en el escenario anterior pero para el rival

- anti_recompensa: Si jugando esa acción, le dejo la partida ganada al rival, entonces me castiga.


En otro caso se le dará recompensa 0




